In [1]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
from itertools import cycle, islice

In [2]:
# Set Pandas column setting to maximum to display all the columns
pd.set_option("display.max_columns", None)

# Set directory path for the data files
folder_path = "/Users/andre/Data Science/Semester_2/SL/data"

In [3]:
# Define list with variables to be selected from the input files 
selected_variables = [
                      "Season",
                      "Date",
                      "HomeTeam",
                      "AwayTeam",
                      "FTHG",
                      "FTAG",
                      "FTR",
                      "HTHG",
                      "HTAG",
                      "HTR",
                      "Referee",
                      "HS",
                      "AS",
                      "HST",
                      "AST",
                      "HF",
                      "AF",
                      "HC",
                      "AC",
                      "HY",
                      "AY",
                      "HR",
                      "AR",
                      "B365H",
                      "B365D",
                      "B365A"
                     ]

In [4]:
# Define function to convert different date strings into the same format
def parse_date(date_str):
    """
    Parses a date string into a datetime object.

    This function tries to parse the given date string using two common date formats:
    - %d/%m/%Y (e.g., 16/04/2024)
    - %d/%m/%y (e.g., 16/04/24)

    If the date string does not match either format, the function returns None.

    Parameters:
        date_str (str): The date string to parse.

    Returns:
        datetime.datetime: The parsed datetime object if successful, otherwise None.
    """
    
    try:
        # Parse the date string using the first format
        return datetime.strptime(date_str, "%d/%m/%Y")
    except ValueError:
        try:
            # Parse the date string using the second format
            return datetime.strptime(date_str, "%d/%m/%y")
        except ValueError:
            # Return None if both formats fail 
            return None

In [5]:
# Define empty list to concatenate all data files into a single Pandas DataFrame later
dfs = []

# Loop through all CSV files in the given directory path
for file_name in os.listdir(folder_path):
    
    # Only reading the files that include the match history data from all seasons
    if file_name[:3] == "pl_":
    
        # Read current file path
        file_path = os.path.join(folder_path, file_name)
        
        # Read current file into a Pandas DataFrame
        df = pd.read_csv(file_path)

        # Upper case all characters in the Season column
        df["Season"] = file_name[:-4].upper()

        # Select the subset of the columns of interest
        df_selected = df[selected_variables]

        # Include DataFrame into the list
        dfs.append(df_selected)

# Create one single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Parse the Date column and convert the different formats into the same one
combined_df["Date"] = combined_df["Date"].apply(parse_date)

# Display the results
combined_df

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A
0,PL_15_16,2015-08-08,Bournemouth,Aston Villa,0,1,A,0,0,D,M Clattenburg,11,7,2,3,13,13,6,3,3,4,0,0,2.00,3.6,4.00
1,PL_15_16,2015-08-08,Chelsea,Swansea,2,2,D,2,1,H,M Oliver,11,18,3,10,15,16,4,8,1,3,1,0,1.36,5.0,11.00
2,PL_15_16,2015-08-08,Everton,Watford,2,2,D,0,1,A,M Jones,10,11,5,5,7,13,8,2,1,2,0,0,1.70,3.9,5.50
3,PL_15_16,2015-08-08,Leicester,Sunderland,4,2,H,3,0,H,L Mason,19,10,8,5,13,17,6,3,2,4,0,0,1.95,3.5,4.33
4,PL_15_16,2015-08-08,Man United,Tottenham,1,0,H,1,0,H,J Moss,9,9,1,4,12,12,1,2,2,3,0,0,1.65,4.0,6.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7215,PL_21_22,2022-05-22,Crystal Palace,Man United,1,0,H,1,0,H,M Atkinson,6,10,3,4,12,22,3,6,2,4,0,0,3.00,3.6,2.25
7216,PL_21_22,2022-05-22,Leicester,Southampton,4,1,H,0,0,D,J Moss,12,7,6,2,10,5,3,3,0,1,0,0,1.85,4.0,3.80
7217,PL_21_22,2022-05-22,Liverpool,Wolves,3,1,H,1,1,D,A Taylor,29,7,8,5,6,3,5,3,1,0,0,0,1.14,8.5,15.00
7218,PL_21_22,2022-05-22,Man City,Aston Villa,3,2,H,0,1,A,M Oliver,24,4,5,2,5,11,13,1,0,1,0,0,1.16,8.0,17.00


In [6]:
# Read CSV file including all the team ratings from the video game FIFA
fifaindex_df = pd.read_csv(f"{folder_path}/fifaindex.csv", delimiter=";")

# Define dictionary to parse all team names to have consistent data across the DataFrames
mapping_dict = {
                "Leicester City": "Leicester",
                "Manchester United": "Man United",
                "Manchester City": "Man City",
                "Norwich City": "Norwich",
                "Newcastle United": "Newcastle",
                "Wolverhampton Wanderers": "Wolves",
                "Brighton & Hove Albion": "Brighton",
                "Tottenham Hotspur": "Tottenham",
                "Stoke City": "Stoke",
                "West Bromwich": "West Brom",
                "Swansea City": "Swansea",
                "West Ham United": "West Ham",
                "Leeds United": "Leeds",
                "Wigan Athletic": "Wigan",
                "Birmingham City": "Birmingham",
                "Blackburn Rovers": "Blackburn",
                "Charlton Athletic": "Charlton",
                "Bolton Wanderers": "Bolton",
                "Queens Park Rangers": "QPR",
                "Reading FC": "Reading",
                "Hull City": "Hull",
                "Huddersfield Town": "Huddersfield",
                "Cardiff City": "Cardiff",
                "Derby Country": "Derby",
                "Nottingham Forest": "Nott'm Forest"
               }

# Parse all team names and replace all inconsistent names
fifaindex_df["Team"].replace(mapping_dict, inplace=True)

# Display results
fifaindex_df

,Season,Team,Attack,Midfield,Defense,Overall
0,PL_04_05,Arsenal,92,91,89,90
1,PL_04_05,Man United,91,90,85,88
2,PL_04_05,Chelsea,87,88,86,87
3,PL_04_05,Liverpool,88,86,82,84
4,PL_04_05,Newcastle,85,83,76,81
...,...,...,...,...,...,...
395,PL_23_24,Brentford,75,76,76,76
396,PL_23_24,Nott'm Forest,75,77,75,77
397,PL_23_24,Fulham,76,77,76,77
398,PL_23_24,Luton Town,72,72,72,72


In [7]:
# Read CSV file including the matchweek data
matches_df = pd.read_csv(f"{folder_path}/matches.csv")

# Define dictionary to parse all team names to have consistent data across the DataFrames
mapping_dict = {
                "Birmingham City": "Birmingham",
                "Cardiff City": "Cardiff",
                "Charlton Ath": "Charlton",
                "Derby County": "Derby",
                "Hull City": "Hull",
                "Leicester City": "Leicester",
                "Leeds United": "Leeds",
                "Manchester Utd": "Man United",
                "Manchester City": "Man City",
                "Newcastle Utd": "Newcastle",
                "Norwich City": "Norwich",
                "Nott'ham Forest": "Nott'm Forest",
                "Sheffield": "Sheffield United",
                "Sheffield Utd": "Sheffield United",
                "Stoke City": "Stoke",
                "Swansea City": "Swansea",
                "Wigan Athletic": "Wigan"
               }

# Parse all team names and replace all inconsistent names
matches_df["Home"].replace(mapping_dict, inplace=True)
matches_df["Away"].replace(mapping_dict, inplace=True)

# Convert and parse the Date column and convert the different formats into the same one
matches_df["Date"] = pd.to_datetime(matches_df["Date"])
matches_df["Date"] = matches_df["Date"].dt.strftime("%d/%m/%Y")
matches_df["Date"] = matches_df["Date"].apply(parse_date)

# Select the subset of the columns of interest
matches_df = matches_df[["Wk", "Date", "Home", "Away"]]

# Display results
matches_df

,Wk,Date,Home,Away
0,1,1992-08-15,Coventry City,Middlesbrough
1,1,1992-08-15,Leeds,Wimbledon
2,1,1992-08-15,Sheffield United,Man United
3,1,1992-08-15,Crystal Palace,Blackburn
4,1,1992-08-15,Arsenal,Norwich
...,...,...,...,...
12021,38,2023-05-28,Everton,Bournemouth
12022,38,2023-05-28,Leicester,West Ham
12023,38,2023-05-28,Aston Villa,Brighton
12024,38,2023-05-28,Leeds,Tottenham


In [8]:
# Right join the team ratings data to the home teams
merged_df = pd.merge(combined_df,
                     fifaindex_df,
                     left_on=["Season", "HomeTeam"],
                     right_on=["Season", "Team"], how="left")

# Right join the team ratings data to the away teams
merged_df = pd.merge(merged_df,
                     fifaindex_df,
                     left_on=["Season", "AwayTeam"],
                     right_on=["Season", "Team"], how="left")

# Right join the match week data
merged_df = pd.merge(merged_df,
                     matches_df,
                     left_on=["Date", "HomeTeam", "AwayTeam"],
                     right_on=["Date", "Home", "Away"], how="left")

# Define dictionary to rename columns
rename_dict = {
                "Attack_x": "HAttack",
                "Midfield_x": "HMidfield",
                "Defense_x": "HDefense",
                "Overall_x": "HOverall",
                "Attack_y": "AAttack",
                "Midfield_y": "AMidfield",
                "Defense_y": "ADefense",
                "Overall_y": "AOverall"              
              }

# Rename columns
merged_df.rename(columns=rename_dict, inplace=True)

# Convert Wk column to integer
merged_df["Wk"] = merged_df["Wk"].astype(int)

# Drop some unnecessary columns
merged_df = merged_df.drop(columns=["Team_x", "Team_y", "Home", "Away"])

# Save all column names accept of HomeTeam and AwayTeam in a list
id_vars = [col for col in merged_df.columns if col not in ["HomeTeam", "AwayTeam"]]

# Melt the columns HomeTeam and AwayTeam into the new columns Side and Team
merged_df = merged_df.melt(id_vars=id_vars,
                           value_vars=["HomeTeam", "AwayTeam"],
                           var_name="Side",
                           value_name="Team")

# Sort DataFrame according the columns Season, Team, and Date in ascending order
merged_df = merged_df.sort_values(["Season", "Team", "Date"], ascending=[True, True, True])

# Add a new column Game repeating the sequence 1 to 38 because a season has 38 matchdays
merged_df["Game"] = [*islice(cycle(list(range(1, 39))), len(merged_df))]

# Convert Column Game into string
merged_df["Game"] = merged_df["Game"].astype(str)

# Create a dummy column merging Team and Game with a comma in between
merged_df["merged_dummy"] = merged_df["Team"] + ", " + merged_df["Game"]

# Unmelting the DataFrame again by recreating the columns HomeTeam and AwayTeam and adding the two new columns
# HomeGame and AwayGame because in some cases due to change of schedule two teams could have played a different
# amount of games when they phased each other
merged_df = merged_df.pivot(index=id_vars, columns="Side", values="merged_dummy").reset_index()
merged_df[["AwayTeam", "AwayGame"]] = merged_df["AwayTeam"].str.split(", ", expand=True)
merged_df[["HomeTeam", "HomeGame"]] = merged_df["HomeTeam"].str.split(", ", expand=True)
merged_df["AwayGame"] = merged_df["AwayGame"].astype(int)
merged_df["HomeGame"] = merged_df["HomeGame"].astype(int)

# Display results
merged_df

Side,Season,Date,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,HAttack,HMidfield,HDefense,HOverall,AAttack,AMidfield,ADefense,AOverall,Wk,AwayTeam,HomeTeam,AwayGame,HomeGame
0,PL_04_05,2004-08-14,1,1,D,0,1,A,C Foy,12,4,4,2,15,17,4,5,1,0,0,0,1.80,3.10,5.00,72,78,75,76,78,72,71,73,1,West Brom,Blackburn,1,1
1,PL_04_05,2004-08-14,1,1,D,0,1,A,D Gallagher,14,16,7,8,17,11,3,8,3,1,0,0,3.10,3.20,2.25,81,76,75,77,88,86,82,84,1,Liverpool,Tottenham,1,1
2,PL_04_05,2004-08-14,1,1,D,1,0,H,M Messias,12,4,5,2,14,12,9,4,0,2,0,0,2.10,3.25,3.40,81,76,74,77,78,76,77,78,1,Fulham,Man City,1,1
3,PL_04_05,2004-08-14,1,1,D,1,0,H,P Walton,14,14,10,8,14,16,6,11,0,1,0,0,2.10,3.25,3.40,74,72,73,73,69,68,72,69,1,Crystal Palace,Norwich,1,1
4,PL_04_05,2004-08-14,1,1,D,1,1,D,H Webb,14,16,8,11,13,16,8,4,2,4,0,0,2.40,3.20,2.87,78,73,75,75,80,76,77,77,1,Birmingham,Portsmouth,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7215,PL_22_23,2023-05-28,2,1,H,1,0,H,S Hooper,13,16,4,3,8,10,3,5,1,1,0,0,1.91,3.80,3.60,82,80,76,79,79,80,78,79,38,West Ham,Leicester,38,38
7216,PL_22_23,2023-05-28,2,1,H,1,1,D,R Jones,21,10,8,3,14,10,5,4,1,2,0,0,1.50,4.33,6.00,82,83,81,82,77,76,74,76,38,Fulham,Man United,38,38
7217,PL_22_23,2023-05-28,2,1,H,2,1,H,D Coote,12,8,5,4,15,16,4,3,4,4,0,0,2.00,3.60,3.50,78,79,79,79,75,76,77,76,38,Brighton,Aston Villa,38,38
7218,PL_22_23,2023-05-28,4,4,D,2,2,D,D England,15,30,10,8,4,10,2,9,0,2,0,0,6.50,4.75,1.44,76,72,76,76,84,82,84,84,38,Liverpool,Southampton,38,38


In [9]:
# Extract all seasons
seasons = sorted(merged_df["Season"].unique())

# Set stealing fraction gamma for updating home and away forms
gamma = 0.33

# Create new columns
merged_df["HForm"] = 1.0
merged_df["AForm"] = 1.0
merged_df["HTGC"] = 0
merged_df["ATGC"] = 0
merged_df["HTGS"] = 0
merged_df["ATGS"] = 0

# Iterate through all seasons
for season in seasons:
    
    # Extract data for current season
    season_df = merged_df[merged_df["Season"]==season]
    
    # Extract all teams of current season
    teams = sorted(season_df["HomeTeam"].unique())
    
    # Create a temporary DataFrame to keep track of the latest form, goal scored, and goals conceded for every team
    cur_form_df = pd.DataFrame(teams, columns=["Team"])
    cur_form_df["Form"] = 1.0
    cur_form_df["GC"] = 0
    cur_form_df["GS"] = 0
    
    # Iterate through every matchday
    for i in range(1, 39):
        
        # Iterate through every team
        for team in teams:
            
            # Set a few logical conditions to keep code more readable
            # Condition1: Select data for current season
            # Condition2: Select data for current matchday and team being the home team
            # Condition2: Select data for current matchday and team being the away team
            condition1 = merged_df["Season"]==season
            condition2 = (merged_df["HomeGame"]==i) & (merged_df["HomeTeam"]==team) 
            condition3 = (merged_df["AwayGame"]==i) & (merged_df["AwayTeam"]==team)

            # Select data for current season, matchday, and team
            matchday_df = merged_df[condition1 & (condition2 | condition3)]
            
            # Read names of home and away team and scored goals by each team
            home_team = matchday_df["HomeTeam"].iloc[0]
            away_team = matchday_df["AwayTeam"].iloc[0]
            home_goals = matchday_df["FTHG"].iloc[0]
            away_goals = matchday_df["FTAG"].iloc[0]
            
            # Read current form of home and away team
            home_form = cur_form_df[cur_form_df["Team"]==home_team]["Form"].iloc[0]
            away_form = cur_form_df[cur_form_df["Team"]==away_team]["Form"].iloc[0]
            
            # If it's not the first matchday enter the current form for the home and away team and the total 
            # conceded and scored goals (so far) by both teams into the final DataFrame
            if i != 1:
                if team==home_team:
                    merged_df.loc[condition1 & condition2, "HForm"] = home_form
                    merged_df.loc[condition1 & condition2,
                                  "HTGC"] = cur_form_df[cur_form_df["Team"]==home_team]["GC"].iloc[0]
                    merged_df.loc[condition1 & condition2,
                                  "HTGS"] = cur_form_df[cur_form_df["Team"]==home_team]["GS"].iloc[0]
                elif team==away_team:
                    merged_df.loc[condition1 & condition3, "AForm"] = away_form
                    merged_df.loc[condition1 & condition3,
                                  "ATGC"] = cur_form_df.loc[cur_form_df["Team"]==away_team]["GC"].iloc[0]
                    merged_df.loc[condition1 & condition3,
                                  "ATGS"] = cur_form_df.loc[cur_form_df["Team"]==away_team]["GS"].iloc[0]
            
            # Update the total scored and conceded goals for both teams
            if team==home_team:
                cur_form_df.loc[cur_form_df["Team"]==home_team, "GC"] += away_goals
                cur_form_df.loc[cur_form_df["Team"]==home_team, "GS"] += home_goals
            elif team==away_team:
                cur_form_df.loc[cur_form_df["Team"]==away_team, "GC"] += home_goals
                cur_form_df.loc[cur_form_df["Team"]==away_team, "GS"] += away_goals
            
            # Update the form for both teams depending on the match result
            # H: Home team won
            # A: Away team won
            # D: Draw
            if matchday_df["FTR"].iloc[0] == "H" and team==home_team:
                cur_form_df.loc[cur_form_df["Team"]==home_team, "Form"] = home_form + gamma*away_form
            elif matchday_df["FTR"].iloc[0] == "H" and team==away_team:
                cur_form_df.loc[cur_form_df["Team"]==away_team, "Form"] = away_form - gamma*away_form    
            elif matchday_df["FTR"].iloc[0] == "A" and team==home_team:
                cur_form_df.loc[cur_form_df["Team"]==home_team, "Form"] = home_form - gamma*home_form
            elif matchday_df["FTR"].iloc[0] == "A" and team==away_team:    
                cur_form_df.loc[cur_form_df["Team"]==away_team, "Form"] = away_form + gamma*home_form
            elif matchday_df["FTR"].iloc[0] == "D" and team==home_team:
                cur_form_df.loc[cur_form_df["Team"]==home_team, "Form"] = home_form - gamma*(home_form-away_form)
            elif matchday_df["FTR"].iloc[0] == "D" and team==away_team:
                cur_form_df.loc[cur_form_df["Team"]==away_team, "Form"] = away_form - gamma*(away_form-home_form)

In [10]:
# Extract all seasons
seasons = sorted(merged_df["Season"].unique())

# Set k
k = 5

# Create new columns
merged_df["HSt"] = 0.0
merged_df["ASt"] = 0.0
merged_df["HGKPP"] = 0.0
merged_df["AGKPP"] = 0.0
merged_df["HHTGKPP"] = 0.0
merged_df["AHTGKPP"] = 0.0
merged_df["HSTKPP"] = 0.0
merged_df["ASTKPP"] = 0.0
merged_df["HSKPP"] = 0.0
merged_df["ASKPP"] = 0.0
merged_df["HYKKP"] = 0.0
merged_df["AYKKP"] = 0.0
merged_df["HRKKP"] = 0.0
merged_df["ARKKP"] = 0.0
merged_df["HCKPP"] = 0.0
merged_df["ACKPP"] = 0.0
merged_df["HFKKP"] = 0.0
merged_df["AFKKP"] = 0.0

# Melt the columns HomeTeam and AwayTeam into the new columns Side and Team
melted_df = merged_df.melt(id_vars=["Season",
                                    "HomeGame",
                                    "AwayGame",
                                    "FTR",
                                    "Wk",
                                    "HS",
                                    "AS",
                                    "HST",
                                    "AST",
                                    "HF",
                                    "AF",
                                    "HC",
                                    "AC",
                                    "HY",
                                    "AY",
                                    "HR",
                                    "AR",
                                    "FTHG",
                                    "FTAG",
                                    "HTHG",
                                    "HTAG"],
                           value_vars=["HomeTeam", "AwayTeam"],
                           var_name="Side",
                           value_name="Team")

# Create new column Points based on the match result
# 3 if home or away team won
# 1 if both drew
# 0 if home or away team lost
melted_df["points"] = np.where((melted_df["Side"] == "HomeTeam") & 
                               (melted_df["FTR"] == "H") | 
                               (melted_df["Side"] == "AwayTeam") & 
                               (melted_df["FTR"] == "A"), 3,
                               np.where(melted_df["FTR"] == "D", 1, 0))

# Update Wk column using the value from the HomeGame and AwayGame columns
melted_df.loc[melted_df["Side"] == "HomeTeam", "Wk"] = melted_df["HomeGame"]
melted_df.loc[melted_df["Side"] == "AwayTeam", "Wk"] = melted_df["AwayGame"]

# Iterate through every season
for season in seasons:
    
    # Extract data for current season
    season_df = merged_df[merged_df["Season"]==season]
    
    # Extract all teams of current season
    teams = sorted(season_df["HomeTeam"].unique())
    
    # Iterate through every matchday skipping the first k matchdays
    for i in range(k+1, 39):
        
        # Iterate through every team
        for team in teams:
            
            # Calculate current home streak
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["HomeGame"]==i) &
                          (merged_df["HomeTeam"]==team),
                          "HSt"] = melted_df[(melted_df["Season"]==season) & 
                                             (melted_df["Team"]==team) &
                                             (melted_df["HomeGame"]<i) &
                                             (melted_df["HomeGame"]>=i-k)]["points"].sum()/(3*k)
            
            # Calculate current away streak
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["AwayGame"]==i) &
                          (merged_df["AwayTeam"]==team),
                          "ASt"] = melted_df[(melted_df["Season"]==season) & 
                                             (melted_df["Team"]==team) &
                                             (melted_df["AwayGame"]<i) &
                                             (melted_df["AwayGame"]>=i-k)]["points"].sum()/(3*k)
            
            # Calculate past k home goals
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["HomeGame"]==i) &
                          (merged_df["HomeTeam"]==team),
                          "HGKPP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["HomeGame"]<i) &
                                               (melted_df["HomeGame"]>=i-k)]["FTHG"].sum()/k
            
            # Calculate past k away goals
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["AwayGame"]==i) &
                          (merged_df["AwayTeam"]==team),
                          "AGKPP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["AwayGame"]<i) &
                                               (melted_df["AwayGame"]>=i-k)]["FTAG"].sum()/k
            
            # Calculate past k home half time goals
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["HomeGame"]==i) &
                          (merged_df["HomeTeam"]==team),
                          "HHTGKPP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["HomeGame"]<i) &
                                               (melted_df["HomeGame"]>=i-k)]["HTHG"].sum()/k
            
            # Calculate past k away half time goals
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["AwayGame"]==i) &
                          (merged_df["AwayTeam"]==team),
                          "AHTGKPP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["AwayGame"]<i) &
                                               (melted_df["AwayGame"]>=i-k)]["HTAG"].sum()/k
            
            # Calculate past k home shots on target
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["HomeGame"]==i) &
                          (merged_df["HomeTeam"]==team),
                          "HSTKPP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["HomeGame"]<i) &
                                               (melted_df["HomeGame"]>=i-k)]["HST"].sum()/k
            
            # Calculate past k away shots on target
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["AwayGame"]==i) &
                          (merged_df["AwayTeam"]==team),
                          "ASTKPP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["AwayGame"]<i) &
                                               (melted_df["AwayGame"]>=i-k)]["AST"].sum()/k
            
            # Calculate past k home shots taken
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["HomeGame"]==i) &
                          (merged_df["HomeTeam"]==team),
                          "HSKPP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["HomeGame"]<i) &
                                               (melted_df["HomeGame"]>=i-k)]["HS"].sum()/k
            
            # Calculate past k away shots taken
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["AwayGame"]==i) &
                          (merged_df["AwayTeam"]==team),
                          "ASKPP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["AwayGame"]<i) &
                                               (melted_df["AwayGame"]>=i-k)]["AS"].sum()/k
            
            # Calculate past k home yellow cards
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["HomeGame"]==i) &
                          (merged_df["HomeTeam"]==team),
                          "HYKKP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["HomeGame"]<i) &
                                               (melted_df["HomeGame"]>=i-k)]["HY"].sum()/k
            
            # Calculate past k away yellow cards
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["AwayGame"]==i) &
                          (merged_df["AwayTeam"]==team),
                          "AYKKP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["AwayGame"]<i) &
                                               (melted_df["AwayGame"]>=i-k)]["AY"].sum()/k
            
            # Calculate past k home red cards
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["HomeGame"]==i) &
                          (merged_df["HomeTeam"]==team),
                          "HRKKP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["HomeGame"]<i) &
                                               (melted_df["HomeGame"]>=i-k)]["HR"].sum()/k
            
            # Calculate past k away red cards
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["AwayGame"]==i) &
                          (merged_df["AwayTeam"]==team),
                          "ARKKP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["AwayGame"]<i) &
                                               (melted_df["AwayGame"]>=i-k)]["AR"].sum()/k
            
            # Calculate past k home corners
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["HomeGame"]==i) &
                          (merged_df["HomeTeam"]==team),
                          "HCKPP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["HomeGame"]<i) &
                                               (melted_df["HomeGame"]>=i-k)]["HC"].sum()/k
            
            # Calculate past k away corners
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["AwayGame"]==i) &
                          (merged_df["AwayTeam"]==team),
                          "ACKPP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["AwayGame"]<i) &
                                               (melted_df["AwayGame"]>=i-k)]["AC"].sum()/k
            
            # Calculate past k home freekicks
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["HomeGame"]==i) &
                          (merged_df["HomeTeam"]==team),
                          "HFKKP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["HomeGame"]<i) &
                                               (melted_df["HomeGame"]>=i-k)]["HF"].sum()/k
            
            # Calculate past k away freekicks
            merged_df.loc[(merged_df["Season"]==season) &
                          (merged_df["AwayGame"]==i) &
                          (merged_df["AwayTeam"]==team),
                          "AFKKP"] = melted_df[(melted_df["Season"]==season) & 
                                               (melted_df["Team"]==team) &
                                               (melted_df["AwayGame"]<i) &
                                               (melted_df["AwayGame"]>=i-k)]["AF"].sum()/k

In [11]:
# Calculate home goal difference
merged_df["HTGD"] = merged_df["HTGS"] - merged_df["HTGC"]

# Calculate away goal difference
merged_df["ATGD"] = merged_df["ATGS"] - merged_df["ATGC"]

# Calculate form differential
merged_df["FormDif"] = merged_df["HForm"] - merged_df["AForm"]

# Calculate streak differential
merged_df["StDif"] = merged_df["HSt"] - merged_df["ASt"]

# Calculate past k goals differential
merged_df["GKPP"] = merged_df["HGKPP"] - merged_df["AGKPP"]

# Calculate past k half time goals differential
merged_df["HTGKPP"] = merged_df["HHTGKPP"] - merged_df["HHTGKPP"]

# Calculate past k shots on target differential
merged_df["STKPP"] = merged_df["HSTKPP"] - merged_df["ASTKPP"]

# Calculate past k shots taken differential
merged_df["SKPP"] = merged_df["HSKPP"] - merged_df["ASKPP"]

# Calculate past k yellow cards differential
merged_df["YKKP"] = merged_df["HYKKP"] - merged_df["AYKKP"]

# Calculate past k red cards differential
merged_df["RKKP"] = merged_df["HRKKP"] - merged_df["ARKKP"]

# Calculate past k corners differential
merged_df["CKPP"] = merged_df["HCKPP"] - merged_df["ACKPP"]

# Calculate past k freekicks differential
merged_df["FKKP"] = merged_df["HFKKP"] - merged_df["AFKKP"]

# Calculate attack rating differential
merged_df["AttackDiff"] = merged_df["HAttack"] - merged_df["AAttack"]

# Calculate midfield rating differential
merged_df["MidfieldDiff"] = merged_df["HMidfield"] - merged_df["AMidfield"]

# Calculate defens rating differential
merged_df["DefenseDiff"] = merged_df["HDefense"] - merged_df["ADefense"]

# Calculate overall rating differential
merged_df["OverallDiff"] = merged_df["HOverall"] - merged_df["AOverall"]

# Calculate goal difference differential
merged_df["GoalDiff"] = merged_df["HTGD"] - merged_df["ATGD"]

In [12]:
merged_df[(merged_df["Season"]=="PL_04_05") & 
          ((merged_df["HomeTeam"]=="Arsenal") | (merged_df["AwayTeam"]=="Arsenal"))]

Side,Season,Date,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,HAttack,HMidfield,HDefense,HOverall,AAttack,AMidfield,ADefense,AOverall,Wk,AwayTeam,HomeTeam,AwayGame,HomeGame,HForm,AForm,HTGC,ATGC,HTGS,ATGS,HSt,ASt,HGKPP,AGKPP,HHTGKPP,AHTGKPP,HSTKPP,ASTKPP,HSKPP,ASKPP,HYKKP,AYKKP,HRKKP,ARKKP,HCKPP,ACKPP,HFKKP,AFKKP,HTGD,ATGD,FormDif,StDif,GKPP,HTGKPP,STKPP,SKPP,YKKP,RKKP,CKPP,FKKP,AttackDiff,MidfieldDiff,DefenseDiff,OverallDiff,GoalDiff
9,PL_04_05,2004-08-15,1,4,A,0,2,A,M Riley,9,18,5,14,14,19,0,7,2,1,0,0,6.00,3.40,1.610,72,75,74,75,92,91,89,90,1,Arsenal,Everton,1,1,1.000000,1.000000,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-20,-16,-15,-15,0
19,PL_04_05,2004-08-22,5,3,H,1,1,D,S Dunn,16,9,11,5,8,14,7,5,1,1,0,0,1.36,4.33,9.000,92,91,89,90,82,79,79,80,2,Middlesbrough,Arsenal,2,2,1.330000,1.000000,1,2,4,2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,0.330000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,12,10,10,3
26,PL_04_05,2004-08-25,3,0,H,0,0,D,N Barry,15,3,9,0,11,18,6,5,2,4,0,0,1.28,5.00,10.000,92,91,89,90,72,78,75,76,3,Blackburn,Arsenal,3,3,1.660000,0.670000,4,4,9,3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,-1,0.990000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,13,14,14,6
30,PL_04_05,2004-08-28,1,4,A,0,3,A,G Poll,5,18,5,15,15,5,4,10,1,1,0,0,9.00,4.33,1.360,74,72,73,73,92,91,89,90,4,Arsenal,Norwich,4,4,0.670000,1.881100,5,4,4,12,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,8,-1.211100,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-18,-19,-16,-17,-9
41,PL_04_05,2004-09-11,0,3,A,0,0,D,M Halsey,9,12,7,7,16,10,9,4,2,2,0,0,7.00,3.60,1.533,78,76,77,78,92,91,89,90,5,Arsenal,Fulham,5,5,0.580905,2.102200,7,5,6,16,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,11,-1.521295,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-14,-15,-12,-12,-12
54,PL_04_05,2004-09-18,2,2,D,1,0,H,P Dowd,11,8,6,5,13,17,3,8,1,3,0,0,1.25,5.00,13.000,92,91,89,90,74,79,73,76,6,Bolton,Arsenal,6,6,2.293899,1.292225,5,6,19,9,1.000000,0.666667,2.0,1.0,0.2,0.6,7.4,4.6,10.8,9.4,1.6,0.6,0.0,0.0,5.2,4.8,12.8,12.4,14,3,1.001674,0.333333,1.0,0.0,2.8,1.4,1.0,0.0,0.4,0.4,18,12,16,14,11
60,PL_04_05,2004-09-25,0,1,A,0,1,A,N Barry,11,12,7,9,16,8,8,1,1,0,0,0,6.50,3.50,1.570,81,76,74,77,92,91,89,90,7,Arsenal,Man City,7,7,0.533449,1.963346,6,7,8,21,0.400000,0.866667,1.6,2.4,0.6,0.8,5.4,6.4,10.6,10.0,1.0,2.2,0.0,0.0,5.4,6.4,13.6,12.8,2,14,-1.429897,-0.466667,-0.8,0.0,-1.0,0.6,-1.2,0.0,-1.0,0.8,-11,-15,-15,-13,-12
74,PL_04_05,2004-10-02,4,0,H,1,0,H,M Dean,15,4,9,0,13,15,6,1,3,1,0,0,1.28,5.00,10.000,92,91,89,90,76,79,75,77,8,Charlton,Arsenal,8,8,2.139384,0.771451,7,10,22,8,0.866667,0.533333,1.2,0.2,0.2,0.0,6.8,4.4,10.2,8.2,1.4,1.2,0.0,0.0,6.0,7.8,14.2,11.6,15,-2,1.367934,0.333333,1.0,0.0,2.4,2.0,0.2,0.0,-1.8,2.6,16,12,14,13,17
86,PL_04_05,2004-10-16,3,1,H,2,1,H,G Poll,18,5,15,2,11,22,6,3,2,4,0,0,1.30,5.00,9.500,92,91,89,90,79,74,77,77,9,Aston Villa,Arsenal,9,9,2.393963,0.682348,7,9,26,10,0.866667,0.466667,1.4,1.0,0.4,0.8,6.8,9.2,10.2,15.2,1.6,1.2,0.0,0.0,6.0,7.0,14.6,13.6,19,1,1.711615,0.400000,0.4,0.0,-2.4,-5.0,0.4,0.0,-1.0,1.0,13,17,12,13,18
97,PL_04_05,2004-10-24,2,0,H,0,0,D,M Riley,11,12,5,6,20,23,3,3,2,3,0,0,2.62,3.00,2.750,91,90,85,88,92,91,89,90,10,Arsenal,Man United,10,10,1.278800,2.619138,7,8,9,29,0.600000,0.866667,1.0,1.4,0.2,0.4,7.4,4.6,14.6,8.2,1.2,2.0,0.0,0.0,5.4,3.4,15.4,14.4,2,21,-1.340338,-0.266667,-0.4,0.0,2.8,6.4,-0.8,0.0,2.0,1.0,-1,-1,-4,-2,-19


In [13]:
merged_df.to_csv("final_output.csv", index=False)